<a href="https://colab.research.google.com/github/Spykabore15/students_RAG_project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive and Install Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q \
  langchain \
  langchain-community==0.2.* \
  faiss-cpu \
  sentence-transformers \
  transformers \
  accelerate \
  pypdf==4.* \
  streamlit \
  pyngrok

print("✅ All dependencies installed!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ All dependencies installed!


### Set up ngrok

In [2]:
# Set up ngrok
from pyngrok import ngrok
from google.colab import userdata
import os

# Get ngrok token from secrets
ngrok_token = userdata.get('NGROK_AUTH_KEY')
ngrok.set_auth_token(ngrok_token)


### Streamit app configuration

In [3]:
%%writefile /content/app.py
import streamlit as st
from pathlib import Path
import json
from typing import List, Tuple
import torch

# Langchain imports
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from sentence_transformers import CrossEncoder

# Page config
st.set_page_config(
    page_title="Enhanced Student RAG Chatbot",
    page_icon="🎓",
    layout="wide"
)

# Constants
DATA_PATH = "/content/drive/MyDrive/TRAINING DATA"

# ==================== QUERY DECOMPOSITION ====================

DECOMPOSITION_PROMPT = """You are a query analyzer. Analyze if this question needs to be broken down into sub-questions.

Question: {question}

Rules:
1. If the question is simple and focused, respond with: SIMPLE
2. If the question has multiple parts or requires information from different sources, decompose it into 2-4 sub-questions.

Response format for complex questions:
SUB-QUERY 1: [first sub-question]
SUB-QUERY 2: [second sub-question]
SUB-QUERY 3: [third sub-question] (if needed)

Examples:

Question: "Who is Dr. Sarah Chen?"
Response: SIMPLE

Question: "Which students worked with Dr. Sarah Chen on AI healthcare projects and what were their evaluation scores?"
Response:
SUB-QUERY 1: Which students worked with Dr. Sarah Chen?
SUB-QUERY 2: What AI healthcare projects exist in the database?
SUB-QUERY 3: What were the evaluation scores for these students?

Question: "Compare robotics projects with environmental science projects"
Response:
SUB-QUERY 1: What robotics projects are in the database?
SUB-QUERY 2: What environmental science projects are in the database?

Now analyze this question:
Question: {question}
Response:"""

def decompose_query(llm, question: str) -> List[str]:
    """
    Use LLM to decide if decomposition is needed
    Returns: list of queries (original or decomposed)
    """
    try:
        prompt = DECOMPOSITION_PROMPT.format(question=question)
        response = llm.predict(prompt)

        # Parse response
        if "SIMPLE" in response:
            return [question]

        # Extract sub-queries
        sub_queries = []
        for line in response.split('\n'):
            if line.strip().startswith('SUB-QUERY'):
                # Extract query after the colon
                query = line.split(':', 1)[1].strip()
                if query:
                    sub_queries.append(query)

        # Fallback: if parsing failed, use original
        return sub_queries if sub_queries else [question]
    except Exception as e:
        st.warning(f"Decomposition failed: {e}. Using original query.")
        return [question]

# ==================== RERANKING ====================

class CrossEncoderReranker:
    """Wrapper for cross-encoder reranking"""

    def __init__(self, model_name="cross-encoder/ms-marco-MiniLM-L-6-v2", device="cuda"):
        self.device = device
        self.model = CrossEncoder(model_name, device=device)

    def rerank(self, query: str, documents: List[Document], top_k: int = 4) -> List[Tuple[Document, float]]:
        """
        Rerank documents based on relevance to query

        Args:
            query: User question
            documents: List of Document objects
            top_k: Number of top documents to return

        Returns:
            List of (document, score) tuples, sorted by score
        """
        if not documents:
            return []

        # Prepare pairs for scoring
        pairs = [[query, doc.page_content] for doc in documents]

        # Get relevance scores
        scores = self.model.predict(pairs)

        # Combine documents with scores
        doc_scores = list(zip(documents, scores))

        # Sort by score (descending)
        doc_scores.sort(key=lambda x: x[1], reverse=True)

        return doc_scores[:top_k]

# ==================== ENHANCED RAG PIPELINE ====================

class EnhancedRAGPipeline:
    """Complete RAG pipeline with decomposition and reranking"""

    def __init__(self, vectorstore, llm, reranker, retrieval_k=20, final_k=4):
        self.vectorstore = vectorstore
        self.llm = llm
        self.reranker = reranker
        self.retrieval_k = retrieval_k
        self.final_k = final_k

        # Retriever with higher k for reranking
        self.retriever = vectorstore.as_retriever(
            search_kwargs={"k": retrieval_k}
        )

    def retrieve_documents(self, sub_queries: List[str]) -> List[Document]:
        """
        Retrieve documents for all sub-queries and deduplicate
        """
        all_docs = []

        # Retrieve for each sub-query
        for sub_q in sub_queries:
            docs = self.retriever.get_relevant_documents(sub_q)
            all_docs.extend(docs)

        # Deduplicate by content hash
        unique_docs = []
        seen_content = set()
        for doc in all_docs:
            content_hash = hash(doc.page_content)
            if content_hash not in seen_content:
                unique_docs.append(doc)
                seen_content.add(content_hash)

        return unique_docs

    def generate_answer(self, question: str, context_docs: List[Tuple[Document, float]]) -> str:
        """
        Generate final answer using LLM
        """
        # Format context with sources
        context = "\n\n".join([
            f"[Source {i+1} - Relevance: {score:.2f}]\n{doc.page_content}"
            for i, (doc, score) in enumerate(context_docs)
        ])

        prompt = f"""You are a helpful assistant that answers using only the provided context.
If the answer is not contained in the context, say "I do not know".

Context:
{context}

Question: {question}

Answer (be concise, structured, and cite sources when relevant):"""

        answer = self.llm.predict(prompt)
        return answer

    def query(self, question: str, enable_decomposition: bool = True, enable_reranking: bool = True):
        """
        Full RAG pipeline with all enhancements

        Returns:
            dict with answer, sub_queries, sources, and metadata
        """
        # Step 1: Query Decomposition
        if enable_decomposition:
            sub_queries = decompose_query(self.llm, question)
        else:
            sub_queries = [question]

        # Step 2: Retrieve documents
        retrieved_docs = self.retrieve_documents(sub_queries)

        # Step 3: Rerank documents
        if enable_reranking and retrieved_docs:
            # Use original question for reranking (not sub-queries)
            reranked_docs = self.reranker.rerank(
                query=question,
                documents=retrieved_docs,
                top_k=self.final_k
            )
        else:
            # No reranking, just take top k
            reranked_docs = [(doc, 1.0) for doc in retrieved_docs[:self.final_k]]

        # Step 4: Generate answer
        answer = self.generate_answer(question, reranked_docs)

        return {
            "answer": answer,
            "sub_queries": sub_queries,
            "sources": reranked_docs,
            "num_retrieved": len(retrieved_docs),
            "num_sources": len(reranked_docs)
        }

# ==================== DOCUMENT LOADING (CACHED) ====================

@st.cache_resource
def load_core_pdfs(data_path: Path) -> List[Document]:
    """Load core PDF documents with caching"""
    core_pdfs = [
        data_path / "Projects.pdf",
        data_path / "criteria.pdf",
        data_path / "Mentors.pdf"
    ]

    docs = []
    for pdf in core_pdfs:
        if pdf.exists():
            pages = PyPDFLoader(str(pdf)).load()
            for p in pages:
                p.metadata.update({"source": pdf.name, "category": "core"})
            docs.extend(pages)
    return docs

@st.cache_resource
def load_student_dirs(data_path: Path) -> List[Document]:
    """Load student directories with caching"""
    docs = []
    students_dir = data_path / "students"

    if not students_dir.exists():
        return docs

    for student_dir in students_dir.iterdir():
        if not student_dir.is_dir():
            continue

        # Load student report
        report_pdf = student_dir / "report.pdf"
        if report_pdf.exists():
            pages = PyPDFLoader(str(report_pdf)).load()
            for p in pages:
                p.metadata.update({
                    "source": f"{student_dir.name}/report.pdf",
                    "student": student_dir.name,
                    "category": "student_report"
                })
            docs.extend(pages)

        # Load student summary
        summary_txt = student_dir / "summary.txt"
        if summary_txt.exists():
            tdocs = TextLoader(str(summary_txt), encoding="utf-8").load()
            for d in tdocs:
                d.metadata.update({
                    "source": f"{student_dir.name}/summary.txt",
                    "student": student_dir.name,
                    "category": "student_summary"
                })
            docs.extend(tdocs)

        # Load student metadata
        meta_json = student_dir / "metadata.json"
        if meta_json.exists():
            try:
                meta = json.loads(meta_json.read_text(encoding="utf-8"))
                meta_doc = Document(
                    page_content=json.dumps(meta, ensure_ascii=False, indent=2),
                    metadata={
                        "source": f"{student_dir.name}/metadata.json",
                        "student": student_dir.name,
                        "category": "student_metadata"
                    }
                )
                docs.append(meta_doc)
            except Exception as e:
                st.warning(f"Couldn't parse {meta_json}: {e}")

    return docs

# ==================== MAIN INITIALIZATION (CACHED) ====================

@st.cache_resource
def initialize_enhanced_rag():
    """Initialize the complete enhanced RAG system with caching"""

    # Check GPU availability
    device = "cuda" if torch.cuda.is_available() else "cpu"
    st.info(f"🖥️ Using device: **{device}**")

    if device == "cpu":
        st.warning("⚠️ GPU not detected! This will be slower. Enable GPU in Colab Runtime settings.")

    with st.spinner("📚 Loading documents..."):
        data_path = Path(DATA_PATH)
        raw_docs = load_core_pdfs(data_path) + load_student_dirs(data_path)
        st.success(f"✅ Loaded **{len(raw_docs)}** documents")

    with st.spinner(" Splitting documents into chunks..."):
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks = splitter.split_documents(raw_docs)
        st.success(f"✅ Created **{len(chunks)}** chunks")

    with st.spinner("Loading embedding model..."):
        embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            model_kwargs={'device': device}
        )
        st.success("✅ Embeddings model loaded")

    with st.spinner(" Building vector store..."):
        vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=embeddings
        )
        st.success("✅ Vector store created")

    with st.spinner("🤖 Loading language model (Mistral-7B)..."):
        pipe = pipeline(
            "text-generation",
            model="mistralai/Mistral-7B-Instruct-v0.2",
            device=0 if device == "cuda" else -1,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            max_new_tokens=300,
            temperature=0.1,
            return_full_text=False
        )
        llm = HuggingFacePipeline(pipeline=pipe)
        st.success(f"✅ Language model loaded on **{device}**")

    with st.spinner("🎯 Loading reranker (Cross-Encoder)..."):
        reranker = CrossEncoderReranker(device=device)
        st.success("✅ Reranker loaded")

    # Create enhanced RAG pipeline
    rag = EnhancedRAGPipeline(
        vectorstore=vectorstore,
        llm=llm,
        reranker=reranker,
        retrieval_k=20,
        final_k=4
    )

    st.success(" **Enhanced RAG system initialized!**")

    return rag

# ==================== STREAMLIT UI ====================

def main():
    st.title("🎓 Enhanced Student Project RAG Chatbot")
    st.markdown("""
    Ask questions about student projects, mentors, and evaluation criteria.
    **New Features:** Query decomposition + Advanced reranking
    """)

    # Sidebar configuration
    with st.sidebar:
        st.header("⚙️ Configuration")

        enable_decomposition = st.checkbox(
            "Enable Query Decomposition",
            value=True,
            help="Break complex questions into simpler sub-queries"
        )

        enable_reranking = st.checkbox(
            "Enable Reranking",
            value=True,
            help="Reorder retrieved documents by relevance"
        )

        show_details = st.checkbox(
            "Show Processing Details",
            value=True,
            help="Display sub-queries and relevance scores"
        )

        st.markdown("---")
        st.markdown("### 💡 Example Questions")
        st.markdown("""
        **Simple:**
        - Who are the mentors for robotics projects?
        - What is Dr. Sarah Chen's expertise?

        **Complex (will decompose):**
        - Which students worked with Dr. Chen on AI healthcare projects and what were their scores?
        - Compare robotics projects with environmental science projects
        - List students who can collaborate on Robot and Air Quality projects
        """)

        st.markdown("---")
        st.markdown("### System Info")
        if torch.cuda.is_available():
            st.success(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        else:
            st.error("❌ No GPU detected")

    # Initialize RAG system
    try:
        rag = initialize_enhanced_rag()
    except Exception as e:
        st.error(f"❌ Error initializing RAG system: {e}")
        st.stop()

    # Chat interface
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

            # Show details if available
            if "details" in message and show_details:
                with st.expander("🔍 Processing Details"):
                    st.markdown(message["details"])

            # Show sources if available
            if "sources" in message:
                with st.expander("📚 Source Documents"):
                    st.markdown(message["sources"])

    # Chat input
    if question := st.chat_input("Ask a question about student projects..."):
        # Add user message
        st.session_state.messages.append({"role": "user", "content": question})
        with st.chat_message("user"):
            st.markdown(question)

        # Get response
        with st.chat_message("assistant"):
            with st.spinner("🤔 Processing your question..."):
                try:
                    # Query the RAG system
                    result = rag.query(
                        question=question,
                        enable_decomposition=enable_decomposition,
                        enable_reranking=enable_reranking
                    )

                    # Display answer
                    st.markdown(result["answer"])

                    # Format processing details
                    details_text = ""
                    if show_details:
                        if len(result["sub_queries"]) > 1:
                            details_text += "**🔍 Query Decomposition:**\n"
                            for i, sq in enumerate(result["sub_queries"], 1):
                                details_text += f"{i}. {sq}\n"
                            details_text += "\n"

                        details_text += f"**Retrieval Stats:**\n"
                        details_text += f"- Retrieved: {result['num_retrieved']} documents\n"
                        details_text += f"- After reranking: {result['num_sources']} documents\n"

                        with st.expander("🔍 Processing Details"):
                            st.markdown(details_text)

                    # Format sources
                    sources_text = ""
                    for i, (doc, score) in enumerate(result["sources"], 1):
                        source = doc.metadata.get('source', 'Unknown')
                        category = doc.metadata.get('category', 'N/A')
                        student = doc.metadata.get('student', 'N/A')

                        sources_text += f"**{i}. {source}** (Score: {score:.3f})\n"
                        sources_text += f"   - Category: {category}\n"
                        if student != 'N/A':
                            sources_text += f"   - Student: {student}\n"
                        sources_text += f"   - Content preview: {doc.page_content[:200]}...\n\n"

                    with st.expander("📚 Source Documents"):
                        st.markdown(sources_text)

                    # Add to chat history
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": result["answer"],
                        "details": details_text if show_details else None,
                        "sources": sources_text
                    })

                except Exception as e:
                    error_msg = f"❌ Error generating response: {str(e)}"
                    st.error(error_msg)
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": error_msg
                    })

    # Action buttons in sidebar
    with st.sidebar:
        st.markdown("---")
        col1, col2 = st.columns(2)

        with col1:
            if st.button("🗑️ Clear Chat", use_container_width=True):
                st.session_state.messages = []
                st.rerun()

        with col2:
            if st.button("🔄 Reload System", use_container_width=True):
                st.cache_resource.clear()
                st.rerun()

if __name__ == "__main__":
    main()

Overwriting /content/app.py


### Optional - Pre-build and Save Vector Store

Run this once to save time

In [4]:
from pathlib import Path
import torch
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import json

print(f"GPU Available: {torch.cuda.is_available()}")

DATA_PATH = "/content/drive/MyDrive/TRAINING DATA"
VECTORSTORE_PATH = "/content/drive/MyDrive/vectorstore_enhanced"

def load_all_documents():
    """Load all documents from the data directory"""
    data_path = Path(DATA_PATH)
    docs = []

    # Load core PDFs
    core_pdfs = [
        data_path / "Projects.pdf",
        data_path / "criteria.pdf",
        data_path / "Mentors.pdf"
    ]

    for pdf in core_pdfs:
        if pdf.exists():
            pages = PyPDFLoader(str(pdf)).load()
            for p in pages:
                p.metadata.update({"source": pdf.name, "category": "core"})
            docs.extend(pages)

    # Load student directories
    students_dir = data_path / "students"
    if students_dir.exists():
        for student_dir in students_dir.iterdir():
            if not student_dir.is_dir():
                continue

            # Report PDF
            report_pdf = student_dir / "report.pdf"
            if report_pdf.exists():
                pages = PyPDFLoader(str(report_pdf)).load()
                for p in pages:
                    p.metadata.update({
                        "source": f"{student_dir.name}/report.pdf",
                        "student": student_dir.name,
                        "category": "student_report"
                    })
                docs.extend(pages)

            # Summary TXT
            summary_txt = student_dir / "summary.txt"
            if summary_txt.exists():
                tdocs = TextLoader(str(summary_txt), encoding="utf-8").load()
                for d in tdocs:
                    d.metadata.update({
                        "source": f"{student_dir.name}/summary.txt",
                        "student": student_dir.name,
                        "category": "student_summary"
                    })
                docs.extend(tdocs)

            # Metadata JSON
            meta_json = student_dir / "metadata.json"
            if meta_json.exists():
                try:
                    meta = json.loads(meta_json.read_text(encoding="utf-8"))
                    meta_doc = Document(
                        page_content=json.dumps(meta, ensure_ascii=False, indent=2),
                        metadata={
                            "source": f"{student_dir.name}/metadata.json",
                            "student": student_dir.name,
                            "category": "student_metadata"
                        }
                    )
                    docs.append(meta_doc)
                except Exception as e:
                    print(f"⚠️ Couldn't parse {meta_json}: {e}")

    return docs

# Load documents
print("📚 Loading documents...")
raw_docs = load_all_documents()
print(f"✅ Loaded {len(raw_docs)} documents")

# Split into chunks
print("Splitting documents...")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""]
)
chunks = splitter.split_documents(raw_docs)
print(f"✅ Created {len(chunks)} chunks")

# Create embeddings
print(" Creating embeddings...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': device}
)
print(f"✅ Embeddings model loaded on {device}")

# Build vector store
print("Building vector store (this may take 2-3 minutes)...")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("✅ Vector store created!")

# Save to disk
print(f" Saving vector store to {VECTORSTORE_PATH}...")
vectorstore.save_local(VECTORSTORE_PATH)
print("✅ Vector store saved! You can now skip this cell in future runs.")


GPU Available: True
📚 Loading documents...
✅ Loaded 134 documents
Splitting documents...
✅ Created 287 chunks
 Creating embeddings...


/tmp/ipython-input-389712022.py:102: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Embeddings model loaded on cuda
Building vector store (this may take 2-3 minutes)...
✅ Vector store created!
 Saving vector store to /content/drive/MyDrive/vectorstore_enhanced...
✅ Vector store saved! You can now skip this cell in future runs.


### Run the app with ngrok

In [5]:
import subprocess
import time
import requests

# Kill any existing streamlit processes
!pkill -f streamlit

# Wait a moment for cleanup
time.sleep(2)

# Start Streamlit in background
print("Starting Streamlit...")
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "/content/app.py",
     "--server.port=8501",
     "--server.address=localhost",  # Changed from 0.0.0.0
     "--server.headless=true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to actually start (checking if it's ready)
print("⏳ Waiting for Streamlit to initialize...")
max_retries = 5
for i in range(max_retries):
    try:
        response = requests.get("http://localhost:8501")
        if response.status_code == 200:
            print("✅ Streamlit is ready!")
            break
    except:
        pass
    time.sleep(2)
    print(f"   Attempt {i+1}/{max_retries}...")
else:
    print("❌ Streamlit failed to start. Check logs below:")
    print(streamlit_process.stderr.read().decode())
    raise Exception("Streamlit didn't start properly")

# Now create ngrok tunnel
print(" Creating ngrok tunnel...")
public_url = ngrok.connect(8501)

print("=" * 60)
print(" Enhanced RAG Chatbot is LIVE!")
print("=" * 60)
print(f" Public URL: {public_url}")
print("=" * 60)
print("\n✅ Features Enabled:")
print("  - Query Decomposition (LLM-based)")
print("  - Cross-Encoder Reranking")
print("  - Enhanced Retrieval (k=20 → top 4)")
print("  - Source Attribution with Scores")
print("\n⚠️ Keep this cell running to maintain the connection")
print("=" * 60)

Starting Streamlit...
⏳ Waiting for Streamlit to initialize...
   Attempt 1/5...
✅ Streamlit is ready!
 Creating ngrok tunnel...
 Enhanced RAG Chatbot is LIVE!
 Public URL: NgrokTunnel: "https://unmustered-lacresha-tetratomic.ngrok-free.dev" -> "http://localhost:8501"

✅ Features Enabled:
  - Query Decomposition (LLM-based)
  - Cross-Encoder Reranking
  - Enhanced Retrieval (k=20 → top 4)
  - Source Attribution with Scores

⚠️ Keep this cell running to maintain the connection


### Optional - Test the System Directly in Notebook

In [6]:
# Test individual components before running Streamlit

from transformers import pipeline
from langchain import HuggingFacePipeline
from sentence_transformers import CrossEncoder

# Load models
print("Loading models for testing...")
device = 0 if torch.cuda.is_available() else -1

pipe = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device=device,
    torch_dtype=torch.float16 if device >= 0 else torch.float32,
    max_new_tokens=100,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)

# Test decomposition
print("\n" + "="*60)
print("Testing Query Decomposition")
print("="*60)

test_questions = [
    "Who is Dr. Sarah Chen?",
    "Which students worked with Dr. Chen and what were their scores?",
    "Compare robotics and environmental projects"
]

DECOMP_PROMPT = """You are a query analyzer. Analyze if this question needs to be broken down.

Question: {question}

Rules:
1. If simple, respond with: SIMPLE
2. If complex, decompose into sub-questions.

Response format:
SUB-QUERY 1: [question]
SUB-QUERY 2: [question]

Now analyze:
Question: {question}
Response:"""

for q in test_questions:
    prompt = DECOMP_PROMPT.format(question=q)
    response = llm.predict(prompt)
    print(f"\nQ: {q}")
    print(f"Response: {response[:200]}")

# Test reranker
print("\n" + "="*60)
print("Testing Cross-Encoder Reranker")
print("="*60)

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

query = "AI healthcare projects"
test_docs = [
    "This project focuses on AI-powered medical diagnosis using deep learning.",
    "Students worked on environmental monitoring systems.",
    "The healthcare AI system achieved 95% accuracy in disease prediction.",
    "Robotics project for autonomous delivery in hospitals."
]

pairs = [[query, doc] for doc in test_docs]
scores = reranker.predict(pairs)

print(f"Query: {query}\n")
for doc, score in sorted(zip(test_docs, scores), key=lambda x: x[1], reverse=True):
    print(f"Score: {score:.3f} - {doc[:60]}...")

print("\n✅ All components working! Ready to use Streamlit app.")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading models for testing...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipython-input-1269056518.py:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-1269056518.py:50: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm.predict(prompt)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Testing Query Decomposition


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Q: Who is Dr. Sarah Chen?
Response: 
This question is simple and does not require decomposition.

SIMPLE.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Q: Which students worked with Dr. Chen and what were their scores?
Response: 
SUB-QUERY 1: Identify students who worked with Dr. Chen.
SUB-QUERY 2: Get scores for the identified students.

Q: Compare robotics and environmental projects
Response: 
This question is complex as it involves comparing two distinct areas: robotics and environmental projects.

To effectively compare these two areas, we need to break down the question into sub-queries

Testing Cross-Encoder Reranker
Query: AI healthcare projects

Score: 5.700 - This project focuses on AI-powered medical diagnosis using d...
Score: 2.920 - The healthcare AI system achieved 95% accuracy in disease pr...
Score: -2.748 - Robotics project for autonomous delivery in hospitals....
Score: -11.337 - Students worked on environmental monitoring systems....

✅ All components working! Ready to use Streamlit app.


### Stop Everything and Cleanup


In [7]:
# Run this when you're done

import os
import signal

# Kill streamlit
!pkill -f streamlit

# Kill ngrok tunnels
ngrok.disconnect(public_url)

print("✅ Streamlit and ngrok stopped")

✅ Streamlit and ngrok stopped
